In [188]:
import numpy as np
import random

def isPrime(n, k):
    if n == 1: return False 
    if n == 2: return True
    rootFound = False
    for i in range(k):
        if rootFound: break
        a = random.randint(2,n-1)
        if pow(a, n-1, n) != 1: return False
        # We can write n-1 as t2^r, t odd
        t = n-1
        while t % 2 == 0:
            t >>= 1
        b =pow(a, t, n)
        if b: continue
        while b != -1:
            b = pow(b, 2, n)
            if b == 1: 
                rootFound = True
    return True

def safePrime(l):
    while(1):
        n = random.randint(1<<l, 1<<(l+1))
        if isPrime(n, 5) and isPrime(n//2, 5):
            return n
        
def generator(p):
    while(1):
        g = random.randint(1, p-1)
        if pow(g, (p-1)//2, p) == p-1:
            return g
def key_gen(l):
    p = safePrime(l)
    q = p//2
    g = pow(generator(p),2,p)
    x = random.randint(1, q-1)
    h = pow(g, x, p)
    return ((p,q,g,h),(p,q,g,x))

# Code from GeeksforGeeks
def modInverse(a, m):
    m0 = m
    y = 0
    x = 1
 
    if (m == 1):
        return 0
 
    while (a > 1):
        # q is quotient
        q = a // m
        t = m
        # m is remainder now, process
        # same as Euclid's algo
        m = a % m
        a = t
        t = y
        # Update x and y
        y = x - q * y
        x = t
    # Make x positive
    if (x < 0):
        x = x + m0
    return x

def encrypt(m, ck, r): # change ck to dictionary????
    p,q,g,h = ck
    return (pow(g, r, p), (pow(g, int(m), p) * pow(h,r,p)) % p) # we make sure that m is of type int

def decrypt(c, sk, r): ### change decrypt to recieve randomness?
    p,q,g,x = sk
    gr = pow(g, r, p)
    # remove randomness and get g^M
    g_m = (c * pow(modInverse(gr, p), x, p)) % p
    # solve "easy" DLP by brute-force
    i = 0
    temp = 1
    while(1):
        if temp == g_m: return i
        i += 1
        temp = (temp * g) % p

In [189]:
import names
import pickle
from itertools import permutations
#Setup 
#n Voters, m Candidates
n = 100
m = 4
# Generate random candidate names
P = np.array([names.get_full_name() for i in range(m)])
# Public key ck and private key sk generation given security parameter l
ck, sk = key_gen(100)
p,q,g,h = ck
# Commit and Decommit functions
Com = lambda ck, M, r: encrypt(M, ck, r)
Dec = lambda sk, C, r: decrypt(C, sk, r)
# Choose random permutations for each side of each ballot
perm = np.array([[np.random.permutation(m) for a in range(2)] for i in range(n)])
# Choose unique vote codes
C = np.array([[np.random.permutation(range(2*m*l + a*m + 1, 2*m*l + (a+1)*m + 1)) for a in range(2)] for l in range(1, n+1)])
x = np.max([2*m-1, n+1])
# Create array of powers of x. The (i+1)-th power indicates the (i+1)-th rank/position
R = []
temp = 1
for i in range(m):
    R.append(temp)
    temp *= x
# Commit vote code values and apply permutation
t = np.array([[[random.randint(1,q-1) for j in range(m)] for a in range(2)] for i in range(n)])
U = np.array([[C[i][a][perm[i][a]] for a in range(2)] for i in range(n)])
U = np.array([[[Com(ck, U[i][a][j], t[i][a][j]) for j in range(m)] for a in range(2)] for i in range(n)])
Vrand, U = U[:,:,:,0], U[:,:,:,1]
# Generate double-ballots: s[i] contains the tuple (tag, s0, s1) of voter i
s = list([(i+1, [(P[k], C[i][0][k]) for k in range(m)], [(P[k], C[i][1][k]) for k in range(m)]) for i in range(n)])
# Commit ranks and apply same permutation as before
r = np.array([[[random.randint(1,q-1) for j in range(m)] for a in range(2)] for i in range(n)])
E = np.array([[np.array([Com(ck, R[j], r[i][a][j]) for j in range(m)])[perm[i][a]] for a in range(2)] for i in range(n)])
Rrand, E = E[:,:,:,0], E[:,:,:,1]
# Gather the public info and commit to BB
Publ = [(i+1, U[i], E[i]) for i in range(n)]
Pub = {
    "ck": ck,
    "P": P,
    "n": n, 
    "Publ": Publ
}
with open('BB', 'wb') as fbb:
    pickle.dump(Pub, fbb)

In [190]:
import pickle
# # Saving ballots to 'ballots'
# with open('ballots', 'wb') as f:
#     pickle.dump(s, f)

In [191]:
def productModp(prodList, p = p):
    prod = 1
    for t in prodList:
        prod = (prod * t) % p
    return prod

### tally ###
with open('votes','rb') as fv:
    votes = pickle.load(fv)
for i in range(n):
    votes[i] = (votes[i][0], votes[i][1], list(map(int, votes[i][2])))
print(votes[0])
# Create tuples (vote, audit side of double-ballot) for each voter and commit to BB
auditVS = []
for v in votes:
    auditVS.append((s[v[0] - 1, 1-v[1]], v))
# with open('BB', 'wb') as fbb:
#     pickle.dump(auditVS, fbb)

# Open vote_code committments
Uopen = [[[(Dec(sk, U[i,a,j], t[i,a,j]), t[i,a,j]) for j in range(m)] for a in range(2)]for i in range(n)]
# Open rank committments from the side not voted
Eopen = [Eopen.append(E[v[0]-1,1-v[1]]) for v in votes]
ropen = [r[v[0]-1, 1-v[1]] for v in votes]
### Change how this is done?
Etally = [[] for i in range(m)]
rand = [[] for i in range(m)]
for v in votes:
    # Ord[vote_code] == corresponding rank
    Ord = dict(zip(C[v[0]-1][v[1]], range(m)))
    for j in range(m):
        temp = perm[v[0]-1, v[1], Ord[int(v[2][j])]]
        Etally[j].append(E[v[0]-1, v[1], temp])
        rand[j].append(r[v[0]-1, v[1], temp])

Esum = [prodModp(tally) for tally in Etally]
R = [sum(randList) for randList in rand]
T = [Dec(sk, Esum[i], R[i]) for i in range(m)]
Open = [[(Dec(sk, Eopen[i,j], ropen[i,j]), ropen[i,j]) for j in range(m)] for i in range(n)]

pub = {
    "Pub": Pub,
    "openBallots": auditVS,
    "Uopen": Uopen,
    "Result": (Etally, Esum, (T,R))
    "Eopen": (Open, Eopen)
}

with open('BB', 'wb') as fbb:
    pickle.dump(pub, fbb)

(1, 1, [14, 13, 16, 15])
HELLO
[[[12, 9, 11, 10], [15, 13, 16, 14]], [[18, 17, 19, 20], [24, 22, 23, 21]], [[28, 25, 27, 26], [31, 32, 29, 30]], [[35, 33, 34, 36], [38, 37, 39, 40]], [[43, 42, 44, 41], [46, 45, 47, 48]], [[51, 49, 50, 52], [53, 56, 54, 55]], [[58, 57, 60, 59], [64, 62, 63, 61]], [[67, 65, 66, 68], [72, 70, 71, 69]], [[76, 74, 73, 75], [77, 80, 78, 79]], [[81, 84, 83, 82], [85, 86, 87, 88]], [[91, 90, 92, 89], [95, 93, 96, 94]], [[99, 97, 100, 98], [103, 104, 102, 101]], [[108, 107, 106, 105], [110, 111, 109, 112]], [[113, 115, 116, 114], [118, 120, 119, 117]], [[122, 123, 121, 124], [126, 125, 127, 128]], [[130, 129, 132, 131], [134, 133, 135, 136]], [[138, 140, 137, 139], [141, 142, 143, 144]], [[145, 147, 148, 146], [152, 150, 151, 149]], [[153, 156, 154, 155], [159, 157, 158, 160]], [[163, 162, 161, 164], [167, 168, 165, 166]], [[170, 171, 172, 169], [173, 175, 176, 174]], [[179, 180, 177, 178], [181, 182, 184, 183]], [[188, 187, 185, 186], [190, 189, 192, 191]], [[

In [192]:
# ### result ###
# # given decommitted value of result (polynomial of x of order m) compute res = [s_0,s_1,...,s_{m-1}] where
# # s_i is the number of times that this candidate was given a rank of i.
# def produceResult(decommitment):
#     res = []
#     T = decommitment
#     for j in range(m):
#         s = T % x
#         res.append(s)
#         T = (T - s) // x
#     return res
# def discreteLog(g, h, q, p):
#     k = int(np.ceil(np.sqrt(n*pow(x,m-1))))
    
# A = np.stack((Rand[0], Etally[0]), axis=1)
# Dec(sk, (np.prod(Rand[0]) % p,np.prod(Etally[0])%p))

In [193]:
# ####
# {
#     ck: ,
#     P: [candidates],
#     Pub: [(tagl, Ul, El)],
#      
# }

In [194]:
np.ceil(4.5)

5.0